In [1]:
import os
import tweepy as tw
import pandas as pd
from geotext import GeoText
import nltk
import spacy
import locationtagger
import matplotlib.pyplot as plt
from branca.element import Figure
import folium


In [8]:
consumer_key = "0g9VInDv42k0BJrk5EY2Dlffr"
consumer_secret = "2xCgDvdvKK3JxZ196Gedxd6YnUyBn13Z640N1sbLMvfqdXbWpD"
access_token = "1533339797187883008-A5uEjCSsKsH4OBeTaO8g0vHaqoI8h6"
access_token_secret = "UKvwBMeTRN6p4f2ogcrQAQT52Fi6eWUhe2pCnEkaJ8UGq"

#bear token 

In [9]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [10]:
# Define the search term and the date_since date as variables
search_words = "#wildfire OR #wildfires OR #forestfire OR #bushfire"
date_since = "2018-01-01"

In [5]:
new_search = search_words + " -filter:retweets"
new_search

'#wildfire OR #wildfires OR #forestfire OR #bushfire -filter:retweets'

In [11]:
tweets = tw.Cursor(api.search_tweets,
                       q=new_search,
                       lang="en").items(5000)


In [12]:
all_tweets = [[tweet.text] for tweet in tweets]


In [13]:
tweet_text = pd.DataFrame(data=all_tweets, 
                    columns=["tweet"])
                    
tweet_text

,tweet
0,@BigCatRescue on #LinkedIn.\nhttps://t.co/0HKj...
1,"There was no path or light, only countless vol..."
2,Bushfire Commercialisation Fund: positioning #...
3,Chongqing Wildfire - The strength and power of...
4,"Nothing can stop me, I'm all the way up.\n\n#R..."
...,...
1698,Documenting the destructive force of increasin...
1699,"Retardant drops on the #OwlFire #PleasantFire,..."
1700,Fires to Date\n\nUpdate Detected: 20 Aug 2022 ...
1701,Saskatchewan Daily Wildfire Situation Map\n\nU...


In [15]:
df = tweet_text[tweet_text["tweet"].str.contains("RT") == False]
df

,tweet
0,@BigCatRescue on #LinkedIn.\nhttps://t.co/0HKj...
1,"There was no path or light, only countless vol..."
2,Bushfire Commercialisation Fund: positioning #...
3,Chongqing Wildfire - The strength and power of...
4,"Nothing can stop me, I'm all the way up.\n\n#R..."
...,...
1698,Documenting the destructive force of increasin...
1699,"Retardant drops on the #OwlFire #PleasantFire,..."
1700,Fires to Date\n\nUpdate Detected: 20 Aug 2022 ...
1701,Saskatchewan Daily Wildfire Situation Map\n\nU...


In [16]:
df['tweet'] = df['tweet'].str.title()
df

/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_20994/3597067682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'] = df['tweet'].str.title()


,tweet
0,@Bigcatrescue On #Linkedin.\nHttps://T.Co/0Hkj...
1,"There Was No Path Or Light, Only Countless Vol..."
2,Bushfire Commercialisation Fund: Positioning #...
3,Chongqing Wildfire - The Strength And Power Of...
4,"Nothing Can Stop Me, I'M All The Way Up.\n\n#R..."
...,...
1698,Documenting The Destructive Force Of Increasin...
1699,"Retardant Drops On The #Owlfire #Pleasantfire,..."
1700,Fires To Date\n\nUpdate Detected: 20 Aug 2022 ...
1701,Saskatchewan Daily Wildfire Situation Map\n\nU...


In [17]:

df_countries = pd.DataFrame()


df_regions = pd.DataFrame()


df_cities = pd.DataFrame()


for index, row in df.iterrows():
    countries = row['tweet']
    place_entity = locationtagger.find_locations(text = countries)
    if place_entity.countries:
     df_countries = df_countries.append({'country': place_entity.countries}, ignore_index=True)
    if place_entity.regions:
     df_regions = df_regions.append({'region': place_entity.regions}, ignore_index=True)
    if place_entity.cities:
        for city_name in place_entity.cities:
            df_cities = df_cities.append({'city': city_name}, ignore_index=True)




/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_20994/2338329552.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_countries = df_countries.append({'country': place_entity.countries}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_20994/2338329552.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities = df_cities.append({'city': city_name}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_20994/2338329552.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_regions = df_regions.append({'region': place_entity.regions}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_20994/2338329552.py:19: FutureWarning: The 

In [18]:
df_countries

,country
0,[China]
1,[China]
2,[Portugal]
3,[Canada]
4,[Canada]
...,...
136,[Peru]
137,[Argentina]
138,[Spain]
139,[China]


In [19]:
df_regions


,region
0,[Oregon]
1,[Utah]
2,[Saskatchewan]
3,[Saskatchewan]
4,[California]
...,...
268,[Virginia]
269,[Saskatchewan]
270,[California]
271,[Saskatchewan]


In [20]:
df_cities

df_cities.drop_duplicates(subset=['city'], keep=False)

,city
3,Ellenville
4,Mori
6,Reno
7,Energy
9,Chilliwack
...,...
556,Peru
563,Josephine
567,Virginia
570,Lake


In [21]:
from geopy.geocoders import Nominatim

loc = Nominatim(user_agent="cal")
print(loc)


In [22]:
df_cities_map = pd.DataFrame()

for ind in df_cities.index:
    getloc = loc.geocode(df_cities['city'][ind])
    df_cities_map = df_cities_map.append({'city': df_cities['city'][ind],'latitude': getloc.latitude, 'longitude': getloc.longitude}, ignore_index=True)

df_cities_map=df_cities_map.dropna()
df_cities_map

/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_20994/2917749237.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities_map = df_cities_map.append({'city': df_cities['city'][ind],'latitude': getloc.latitude, 'longitude': getloc.longitude}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_20994/2917749237.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities_map = df_cities_map.append({'city': df_cities['city'][ind],'latitude': getloc.latitude, 'longitude': getloc.longitude}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_20994/2917749237.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities_map = df_cities_map.append({'city': df_ci

,city,latitude,longitude
0,China,35.000074,104.999927
1,Oregon,43.979280,-120.737257
2,Us,39.783730,-100.445882
3,Ellenville,41.717038,-74.395710
4,Mori,45.852444,10.978275
...,...,...,...
567,Virginia,37.123224,-78.492772
568,Us,39.783730,-100.445882
569,California,36.701463,-118.755997
570,Lake,39.050541,-122.777656


In [31]:
df_cities_map.to_csv("city_w_coordinates.csv", sep='\t', encoding='utf-8',index = True, header=True)

In [27]:
# Creating Basemap
fig3=Figure(width=550,height=350)
m3=folium.Map(location=[-1.286389, 36.817223],tiles='cartodbpositron',zoom_start=1)
fig3.add_child(m3)

loc = 'Forest fire related 5000 tweets all around the world'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m3.get_root().html.add_child(folium.Element(title_html))
#Adding markers to the map
for index, row in df_cities_map.iterrows():
    folium.Marker(location=[row['latitude'],row['longitude']],tooltip=row['city'],
        icon=folium.Icon(color="red", icon_color="white", icon="fire")).add_to(m3)


m3



In [33]:
from geopy.exc import GeocoderTimedOut

def put_markers(map, df_cities):
    geo_locator = Nominatim(user_agent="ForestFire")
    for index, row in df_cities_map.iterrows():
        if row['city']:
            try:
                location = geo_locator.geocode(row['city'])
            except GeocoderTimedOut:
                continue
            if location:
                folium.Marker([location.latitude, location.longitude],tooltip=row['city'],
        icon=folium.Icon(color="red", icon_color="white", icon="fire")).add_to(map)



In [34]:

map = folium.Map(location=[0, 0], zoom_start=2)
loc = 'Forest fire related 5000 tweets all around the world'
map.get_root().html.add_child(folium.Element(title_html))
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   
location_data = df_cities
put_markers(map, location_data)
map.save("index.html")